In [3]:
from azure_open_ai import create_metadata, nlp_to_sql_translator
from sql_helper import db_upload, db_execute_sql
import pandas as pd

In [4]:
#Data:
input_data = pd.read_excel(r'I:\12-Data Management\z_Ozturk\Hackhaton\TFA_annon_GPT_dataset.xlsx')
input_data.columns = [col.replace('_', '').replace('-', '') for col in input_data.columns]

#Metadata as "string"
metadata_str = create_metadata(r'I:\12-Data Management\z_Ozturk\Hackhaton\GPT_TMD.xlsx')

In [11]:
metadata_xlsx = pd.read_excel(r'I:\12-Data Management\z_Ozturk\Hackhaton\GPT_TMD.xlsx')
tables = metadata_xlsx['TableName'].unique()
rows = []
for table in tables:
    rows.append((table, metadata_xlsx[metadata_xlsx['TableName'] == table]['ColumnName'].values.tolist()))

print('Unique rows in the metadata table: ', rows)

Unique rows in the metadata table:  [('dimCalendar', ['DateKey', 'Year', 'Quarter', 'Month']), ('dimCountry', ['CountryKey', 'CountryName', 'CountryRegion', 'CountryFlag']), ('dimEntity', ['EntityKey', 'EntityName', 'EntitySubgroup', 'EntityLocalCurrency']), ('dimAccount', ['AccountKey', 'AccountNumber', 'AccountDescription', 'AccountMapping', 'AccountType']), ('dimTaxType', ['TaxTypeKey', 'TaxTypeCategory', 'TaxTypeSubCategory', 'BorneOrCollected']), ('factTable', ['TransactionID', 'JournalID', 'JournalDescription', 'PostingDate', 'CountryKey', 'EntityKey', 'AmountLC', 'AmountRC', 'ReportingCurrency', 'AccountKey', 'TaxTypeKey'])]


In [12]:
dfs = {}
print("All columns in data: ", input_data.columns.values.tolist())
for row in rows:
    columns_to_split_existing = [col for col in row[1] if col in input_data.columns]
    if len(columns_to_split_existing) > 0:
        print('Columns that exists in the metadata table: ', row[0], columns_to_split_existing)
        dfs[row[0]] = input_data[columns_to_split_existing]


All columns in data:  ['AIndex', 'Subgroup', 'CompanyCode', 'Country', 'FinancialYear', 'ReportingYear', 'ReportingQuarter', 'ReportingPeriod', 'AccountNumber', 'AccountName', 'JournalID', 'JournalDescription', 'PostingDate', 'AmountLC', 'LocalCurrency', 'AmountRC', 'ReportingCurrency', 'TFATaxFootprint', 'TFATransactionCategory', 'TFATransactionSubcategory']
Columns that exists in the metadata table:  dimAccount ['AccountNumber']
Columns that exists in the metadata table:  factTable ['JournalID', 'JournalDescription', 'PostingDate', 'AmountLC', 'AmountRC', 'ReportingCurrency']


In [13]:
for idx, table_name in enumerate(list(dfs.keys())):
    print(f'Iteration {idx}')
    #Column list:
    column_list = dfs[table_name].columns.values.tolist()
    
    #Unique items
    print('Unique items: ', set(dfs[table_name].dtypes.values.tolist()))

    #Find Data Types:
    _dict = {'int64': 'int', 'object': 'varchar(250)', 'datetime64[ns]': 'datetime2', 'float64': 'real'}
    types_list = []
    for key in dfs[table_name].dtypes.values.tolist():
        if str(key) in _dict:
            types_list.append(_dict[str(key)])

    column_list_with_types = [f"{col} {type}" for col, type in zip(column_list, types_list)]
    print(column_list_with_types)

    #DB Upload
    db_upload(dfs[table_name], column_list, column_list_with_types, table_name, 'PyAutomate', 'DK2CPHDM01\DM01')    

Iteration 0
Unique items:  {dtype('O')}
['AccountNumber varchar(250)']
Iteration 1
Unique items:  {dtype('<M8[ns]'), dtype('O'), dtype('float64')}
['JournalID varchar(250)', 'JournalDescription varchar(250)', 'PostingDate datetime2', 'AmountLC real', 'AmountRC real', 'ReportingCurrency varchar(250)']


In [18]:
#Create SQL script:
prompt = "What is the lowest transaction amount in local currency?"
response = nlp_to_sql_translator(f"This is my metadata tabel: {metadata_str}. Please create a SQL query for this request {prompt}")
print(response)

['sql\nSELECT MIN(AmountLC) AS LowestTransactionAmountLC\nFROM factTable;\n']


In [19]:
sql_script = response[0].replace('\n', ' ')[3:]
print(sql_script)

 SELECT MIN(AmountLC) AS LowestTransactionAmountLC FROM factTable; 


In [20]:
output_data = db_execute_sql(database_name='PyAutomate', server_name='DK2CPHDM01\DM01', sql_script=sql_script)

In [23]:
print(output_data[0][0])

-9978052608.0
